# 模型热更新
https://www.lanqiao.cn/courses/40981/learning/?id=2813912&compatibility=false

<mark>ERROR</mark>: 
* from thearding import Lock  model_lock = Lock() 在互斥操作之间with model_lock: 进行然后进行操作
* 将某数据类型转化为 List .tolist() ndarray可以
* 注意使用 pickle 的打开方式为'rb' bin文件
* 响应体 直接return 返回 ，对于复杂的 可以使用jsonify({'A':123, 'B':234})

from flask import Flask, request, jsonify 

https://cloud.tencent.com/developer/article/1546921 

非常方便的 jsonify(字典) 将字典转为可以返回的json格式
---
# 手打答案
```python
from flask import Flask, request, jsonify

from threading import Lock
import pickle

app = Flask(__name__)
model_lock = Lock()
# Flask 应用在启动时，读取 `svm_1.pkl` 作为初始模型。
current_model = 'svm_1.pkl'
with open(current_model,'rb') as f:
    model = pickle.load(f)
    
@app.route('/predict', methods=['POST'])
def predict():
    with model_lock:
        inputs = request.get_json()['input']
        output = model.predict(inputs).tolist()

        return jsonify({'current_model':current_model, 'output': output})
    
@app.route('/update', methods=['POST'])
def update():
    with model_lock:
        new_model = request.get_json()['model']
        if new_model in ['svm_1.pkl', 'svm_2.pkl', 'svm_3.pkl']:
            global current_model, model
            current_model = new_model
            with open(current_model, 'rb') as f:
                model = pickle.load(f)
            return current_model, 200

        return 'Invalid model', 400


if __name__ == '__main__':
    app.run()
```